In [28]:
!pip install pandas

You are using pip version 9.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [29]:
import pandas as pd

def read_df(region):
    return pd.read_pickle(path='./tweets_database_{}.pickle'.format(region))

In [30]:
def create_nodes(df):

    # create a DF of nodes of the network
    columns = ['user_id', 'username']
    nodelist = pd.DataFrame(columns=columns)

    # insert all tweet.users
    nodelist['user_id'] = df['user_id_str']
    nodelist['username'] = df['user_screen_name']
    nodelist.set_index('user_id')

    def get_users(user_id, username):
        """Get users from mentions, replies, retweets"""
        reduce = df.dropna(subset=[user_id, username])
        insert_list = []
        for index, row in reduce.iterrows():
            user = {'user_id': row[user_id], 'username': row[username]}
            insert_list.append(user)
        return pd.DataFrame(insert_list, columns=columns)

    # insert all users of tweet.user_mentions
    nodelist = nodelist.append(get_users('user_mention_id_str', 'user_mention_screen_name'), sort=False)

    # insert all users of tweet.replies
    nodelist = nodelist.append(get_users('in_reply_to_user_id_str', 'in_reply_to_screen_name'), sort=False)

    # insert all users of tweet.retweets
    nodelist = nodelist.append(get_users('retweeted_user_id_str', 'retweeted_user_screen_name'), sort=False)

    # drop duplicates
    nodelist.drop_duplicates(subset=columns, inplace=True)
    print(nodelist.info)
    return nodelist

In [31]:
def create_edges(df):

    # Create a DF of directed edges connecting the nodes of the network
    columns = ['source_id', 'target_id']
    edgelist = pd.DataFrame(columns=columns)

    def get_interactions(source_id, target_id):
        """Get interactions from mentions, replies, retweets"""
        reduce = df.dropna(subset=[source_id, target_id])
        insert_list = []
        for index, row in reduce.iterrows():
            interaction = {'source_id': row[source_id], 'target_id': row[target_id]}
            insert_list.append(interaction)
        return pd.DataFrame(insert_list, columns=columns)

    # insert all user mentions, source=original, target=user mentioning another user
    edgelist = edgelist.append(get_interactions(source_id='user_mention_id_str', target_id='user_id_str'))

    # insert all replies, source=original, target=user replying to another user
    edgelist = edgelist.append(get_interactions(source_id='in_reply_to_user_id_str', target_id='user_id_str'))

    # insert all retweets, source=original, target=user retweeting another user
    edgelist = edgelist.append(get_interactions(source_id='retweeted_user_id_str', target_id='user_id_str'))

    # drop duplicates
    edgelist.drop_duplicates(subset=['source_id', 'target_id'], inplace=True)

    # remove self-references (e.g. replies to your own account)
    edgelist = edgelist[edgelist.source_id != edgelist.target_id]

    print(edgelist.info)
    return edgelist

In [32]:
def drop_isolates(nodelist, edgelist):
    # drop users that do not have any retweets, replies, or mentions to keep the graph lighter
    user_edges = edgelist['source_id']
    user_edges = user_edges.append(edgelist['target_id'])
    print(user_edges.shape)
    print(user_edges.drop_duplicates().shape)
    nodelist_filtered = nodelist[nodelist.user_id.isin(list(user_edges))]
    return nodelist_filtered
    

In [33]:
def to_gephi(region, nodelist, edgelist):
    gephi_nodes = nodelist_filtered.rename(columns={'user_id': 'Id', 'username': 'Label'})
    gephi_nodes.to_csv('nodelist_{}.csv'.format(region), index=False)
    gephi_edges = edgelist.rename(columns={'source_id': 'Source', 'target_id': 'Target'})
    gephi_edges.to_csv('edgelist_{}.csv'.format(region), index=False)

In [34]:
def run_for_country(region):
    print("Region: {}".format(region))
    df = read_df(region=region)
    nodelist = create_nodes(df)
    edgelist = create_edges(df)
    
    nodelist = drop_isolates(nodelist, edgelist)
    
    to_gephi(region, nodelist, edgelist)
    

In [35]:
if __name__ == '__main__':
    
    run_for_country(region='italy')
    run_for_country(region='germany')

Region: italy
<bound method DataFrame.info of                     user_id         username
0                 342818725    maggioantonio
1                 521546777          NoveseM
2       1035166905424863233    Malvy89611841
3       1096491048862449664  giovannaconfal4
4                 304695298    WolfgangDanso
5        885567061576908800          yxnggao
6                 427424760         byluke69
8                1251089251  salidaparallela
9                2584588979     AuroreItalie
10                 36039763   Brigante_Paolo
11       785129133009494021     thewaterflea
12      1077319297511374849   Simona53313767
13      1126426788635324417    Romav86737841
14                210749911        biche1780
15                 71242539        WheresMAE
16                336724931          ZanRoby
17       912630324206690305        Damare284
18                235918702    diodeglizilla
19      1222548747130982400    Felix43153007
20                209084129  FRANCESC0PAGANO
21       

NameError: name 'nodelist_filtered' is not defined

<bound method DataFrame.info of                     user_id         username
0                 342818725    maggioantonio
1                 521546777          NoveseM
2       1035166905424863233    Malvy89611841
3       1096491048862449664  giovannaconfal4
4                 304695298    WolfgangDanso
5        885567061576908800          yxnggao
6                 427424760         byluke69
8                1251089251  salidaparallela
9                2584588979     AuroreItalie
10                 36039763   Brigante_Paolo
11       785129133009494021     thewaterflea
12      1077319297511374849   Simona53313767
13      1126426788635324417    Romav86737841
14                210749911        biche1780
15                 71242539        WheresMAE
16                336724931          ZanRoby
17       912630324206690305        Damare284
18                235918702    diodeglizilla
19      1222548747130982400    Felix43153007
20                209084129  FRANCESC0PAGANO
21                 5039

(51338, 2)
(150752,)
(46657,)
(46665, 2)
